In [2]:
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup


In [6]:
data = pd.read_excel('Zipcode_data.xlsx',converters={'ZipCode': lambda x: str(x), 'CountyFIPS': lambda x: str(x), 'StateFIPS': lambda x: str(x)})

In [7]:
len(data)

40

In [8]:
data.head()

,Id,ZipCode,CityName,CityType,CountyName,CountyFIPS,StateName,StateAbbr,StateFIPS
0,91313,20588,DHS,N,HOWARD,27,Maryland,MD,24
1,91319,20598,DHS,P,LOUDOUN,107,Virginia,VA,51
2,92825,24205,BRISTOL,N,BRISTOL CITY,520,Virginia,VA,51
3,96170,31144,KENNESAW,P,COBB,67,Georgia,GA,13
4,96175,31169,PEACHTREE CITY,N,FAYETTE,113,Georgia,GA,13


In [9]:
# testing
url = 'https://www.zip-codes.com/zip-code/09316/zip-code-09316.asp'
print(url)
website_url = requests.get(url)

print(website_url.status_code)   # This should print 200


https://www.zip-codes.com/zip-code/09316/zip-code-09316.asp
200


In [15]:
#this loop will iterrate over each row and use zip code for extracting data
lst = []
for i,j in data.iterrows():
    
    temp_row =[]
    
    zc =j['ZipCode']
    cn = j['CityName']
    ct =  j['CityType']
    con = j['CountyName'] 
    sa = j['StateName']
    sp = j['StateAbbr']
    sf = j ['StateFIPS']
    
    
    temp_row.append(zc)
    temp_row.append(cn)
    temp_row.append(ct)
    temp_row.append(con)
    temp_row.append(sa)
    temp_row.append(sp)
    temp_row.append(sf)
    
    zipcode=zc
    #append 0 so that every zip code length is 5
    if len(zipcode) == 3:
        zipcode = '00'+zipcode
        
    elif len(zipcode) == 4:
        zipcode = '0'+zipcode
        
    #website which will be use to scrap data of a zip code
    url = 'https://www.zip-codes.com/zip-code/'+str(zipcode) + '/zip-code-'+str(zipcode) +'.asp'
    print(url)
    website_url = requests.get(url)

    print(website_url.status_code)   # This should print 200
    if(website_url.status_code == 200):
        soup = BeautifulSoup(website_url.content, 'html.parser')

        gdp_table = soup.find("table", attrs={"class": "statTable"})
        
        if gdp_table is None:
            print('hi')
        else:
            rows = gdp_table.find_all('tr')
            for row in rows:

                count = 0
                tds = list(row.find_all('td'))

                for ele in tds:

                    if ele.text == 'Latitude:':
                        #print(ele.text)
                        lat = ele.text
                        val = tds[1].text
                        print(lat, val)
                        temp_row.append(val)
                    elif ele.text == 'Longitude:':
                        #print(ele.text)
                        long = ele.text
                        val = tds[1].text
                        print(long, val)
                        temp_row.append(val)
            tup = tuple(temp_row)
            lst.append(tup) # append each tuple or rows in list
            time.sleep(20)
    

https://www.zip-codes.com/zip-code/20588/zip-code-20588.asp
200
Latitude: 0
Longitude: 0
https://www.zip-codes.com/zip-code/20598/zip-code-20598.asp
200
Latitude: 38.877442
Longitude: -77.2338
https://www.zip-codes.com/zip-code/24205/zip-code-24205.asp
200
Latitude: 0
Longitude: 0
https://www.zip-codes.com/zip-code/31144/zip-code-31144.asp
200
Latitude: 0
Longitude: 0
https://www.zip-codes.com/zip-code/31169/zip-code-31169.asp
200
Latitude: 0
Longitude: 0
https://www.zip-codes.com/zip-code/34249/zip-code-34249.asp
200
Latitude: 0
Longitude: 0
https://www.zip-codes.com/zip-code/35270/zip-code-35270.asp
200
Latitude: 0
Longitude: 0
https://www.zip-codes.com/zip-code/46288/zip-code-46288.asp
200
Latitude: 0
Longitude: 0
https://www.zip-codes.com/zip-code/50099/zip-code-50099.asp
200
Latitude: 42.0565
Longitude: -93.88
https://www.zip-codes.com/zip-code/50983/zip-code-50983.asp
200
Latitude: 0
Longitude: 0
https://www.zip-codes.com/zip-code/55131/zip-code-55131.asp
200
Latitude: 0
Longitud

In [ ]:
scrap_data= pd.DataFrame(lst, columns=['ZipCode', 'CityName', 'CityType', 'CountyName','StateName', 'StateAbbr', 'StateFIPS', 'Lat', 'Lng'])
scrap_data.to_excel('scrap_data.xlsx', index=False)